In [389]:
import pandas as pd
import numpy as np
import folium
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import datetime
import requests
import geopandas as gpd
from datetime import timedelta

In [390]:
df = pd.read_csv('Locatievoorstellen.csv', header=1, index_col=0)

In [391]:
pd.set_option('display.max_columns', None)

In [392]:
df.columns = df.columns.str.replace(' - ', '_')
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.lower()

In [393]:
df["tijdstip_van_laatste_wijziging"] = pd.to_datetime(df["tijdstip_van_laatste_wijziging"])
df["tijdstip_aangemaakt"] = pd.to_datetime(df["tijdstip_aangemaakt"])
df["tijdstip_locatievoorstel"] = pd.to_datetime(df["tijdstip_locatievoorstel"])
df["tijdstip_controle_locatievoorstel"] = pd.to_datetime(df["tijdstip_controle_locatievoorstel"])
df["tijdstip_advies_exploitant"] = pd.to_datetime(df["tijdstip_advies_exploitant"])
df["tijdstip_publicatie_verkeersbesluit"] = pd.to_datetime(df["tijdstip_publicatie_verkeersbesluit"])
df["tijdstip_aanvraag_netaansluiting"] = pd.to_datetime(df["tijdstip_aanvraag_netaansluiting"])
df["tijdstip_definitief_verkeersbesluit"] = pd.to_datetime(df["tijdstip_definitief_verkeersbesluit"])
df["tijdstip_accepteren_mra-e"] = pd.to_datetime(df["tijdstip_accepteren_mra-e"])
df["tijdstip_realisatie_plannen"] = pd.to_datetime(df["tijdstip_realisatie_plannen"])
df["tijdstip_locatie_voorbereiden"] = pd.to_datetime(df["tijdstip_locatie_voorbereiden"])
df["tijdstip_in_bedrijf"] = pd.to_datetime(df["tijdstip_in_bedrijf"])
df["tijdstip_opleveren_laadpaal"] = pd.to_datetime(df["tijdstip_opleveren_laadpaal"])
df["tijdstip_opleveren_locatie"] = pd.to_datetime(df["tijdstip_opleveren_locatie"])

In [394]:
df['doorlooptijd_d'] = (df['tijdstip_opleveren_locatie'] - df['tijdstip_aangemaakt'])

In [395]:
df = df.drop(columns=['stilgezet_uitleg','verplaatsing_mra-e_laadpaal_id', 'opmerkingen', 'toelichting_bij_verplaatsing', 
                      'locatiecode', 'aanbodgestuurd_opmerkingen', 'datagestuurd_opmerkingen', 'gemeente_akkoord_met_offerte_verplaatsing',
                     'extra_laadpaal_ids', 'referenties', 'aanleiding'])

In [396]:
df['postcode'] = df['postcode'].str.replace('[a-zA-Z]', '', regex=True)
df['postcode'] = df['postcode'].str.replace(' ', '', regex=True)

In [397]:
df2 = df.loc[(df['tijdstip_locatievoorstel'].isna()) | (df['tijdstip_publicatie_verkeersbesluit'].isna()) |
            (df['tijdstip_aanvraag_netaansluiting'].isna()) | (df['tijdstip_definitief_verkeersbesluit'].isna()) |
            (df['tijdstip_opleveren_locatie'].isna()), 'gemeente_akkoord'] = False

df2 = df.loc[(df['tijdstip_advies_exploitant'].isna()), 'exploitant_akkoord'] = False

df2 = df.loc[(df['tijdstip_controle_locatievoorstel'].isna()) | 
             (df['tijdstip_accepteren_mra-e'].isna()) | 
             (df['tijdstip_realisatie_plannen'].isna()) |
             (df['tijdstip_locatie_voorbereiden'].isna()) | 
             (df['tijdstip_in_bedrijf'].isna()) | 
             (df['tijdstip_opleveren_laadpaal'].isna()), 
             'aannemer_akkoord'] = False

In [398]:
df['datum_oplevering_y'] = pd.DatetimeIndex(df['datum_oplevering']).year
df['datum_oplevering_y'] = df['datum_oplevering_y'].fillna(0).astype(int)
df.loc[(df['datum_oplevering_y'] == 0), 'datum_oplevering_y'] = None

In [399]:
df = df.loc[(df['datum_oplevering_y'] < 2024)]

In [400]:
gdf = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df['locatie_longitude'], df['locatie_latitude']))
gdf = gdf.set_crs('EPSG:4326')

In [401]:
gebieden = gpd.read_file('BestuurlijkeGebieden_2023.gml')
gebieden = gebieden.to_crs('EPSG:4326')
prov = gebieden.dissolve(by = 'ligtInProvincieNaam')
prov = prov.reset_index()[['ligtInProvincieNaam','geometry']]
gdf = gdf.sjoin(prov, predicate = 'within')

In [402]:
gdf.rename(columns = {'ligtInProvincieNaam':'provincie'}, inplace = True)

In [403]:
doorlooptijd_df = gdf[['tijdstip_aanvraag_netaansluiting', 'tijdstip_locatie_voorbereiden', 'tijdstip_aangemaakt',
                       'tijdstip_in_bedrijf', 'tijdstip_realisatie_plannen', 'tijdstip_accepteren_mra-e',
                       'provincie', 'locatie_latitude', 'locatie_longitude']].copy()

In [404]:
doorlooptijd_df = doorlooptijd_df.loc[(doorlooptijd_df['tijdstip_locatie_voorbereiden'].notna())]

In [405]:
doorlooptijd_df['aanvraag_voorbereiding'] = (doorlooptijd_df['tijdstip_realisatie_plannen'] - doorlooptijd_df['tijdstip_aanvraag_netaansluiting'])
doorlooptijd_df['voorbereiding_overdracht'] = (doorlooptijd_df['tijdstip_locatie_voorbereiden'] - doorlooptijd_df['tijdstip_accepteren_mra-e'])
doorlooptijd_df['ontvangst_oplevering'] = (doorlooptijd_df['tijdstip_in_bedrijf'] - doorlooptijd_df['tijdstip_accepteren_mra-e'])
doorlooptijd_df['aanvraag_oplevering'] = (doorlooptijd_df['tijdstip_in_bedrijf'] - doorlooptijd_df['tijdstip_aanvraag_netaansluiting'])

In [406]:
doorlooptijd = [doorlooptijd_df['aanvraag_voorbereiding'], doorlooptijd_df['voorbereiding_overdracht'], 
             doorlooptijd_df['ontvangst_oplevering'], doorlooptijd_df['aanvraag_oplevering']]


In [407]:
# Convert to days
seconds_in_day = 86400
for i in range(len(doorlooptijd)):
    doorlooptijd[i] = doorlooptijd[i].dt.total_seconds() / seconds_in_day

In [408]:
doorlooptijd_in_dagen = pd.DataFrame(doorlooptijd).T

In [409]:
doorlooptijd_in_dagen = doorlooptijd_in_dagen[doorlooptijd_in_dagen['aanvraag_oplevering'] > doorlooptijd_in_dagen['aanvraag_voorbereiding']]
doorlooptijd_in_dagen = doorlooptijd_in_dagen[doorlooptijd_in_dagen['aanvraag_oplevering'] > doorlooptijd_in_dagen['voorbereiding_overdracht']]
doorlooptijd_in_dagen = doorlooptijd_in_dagen[doorlooptijd_in_dagen['aanvraag_oplevering'] > doorlooptijd_in_dagen['ontvangst_oplevering']]

In [410]:
doorlooptijd_in_dagen.loc[(doorlooptijd_in_dagen['aanvraag_voorbereiding'] == 0), 'aanvraag_voorbereiding'] = None
doorlooptijd_in_dagen.dropna(inplace = True)

In [411]:
# Drop rows that have 0 days in any of the columns
doorlooptijd_in_dagen = doorlooptijd_in_dagen.loc[(doorlooptijd_in_dagen['aanvraag_voorbereiding'] > 0)]

In [412]:
doorlooptijd_in_dagen['provincie'] = doorlooptijd_df['provincie']

Plotting 

In [413]:
hist_data = [doorlooptijd_in_dagen['aanvraag_voorbereiding'], doorlooptijd_in_dagen['voorbereiding_overdracht'], 
             doorlooptijd_in_dagen['ontvangst_oplevering'], doorlooptijd_in_dagen['aanvraag_oplevering']]
labels = ['aanvraag - voorbereiding', 'voorbereiding - overdracht', 'ontvangst - oplevering', 'aanvraag - oplevering']

fig = ff.create_distplot(hist_data=hist_data, group_labels=labels, bin_size=1, show_rug=False, show_hist=False, show_curve=True, 
                         colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
fig.update_layout(title='Doorlooptijd', xaxis_title='Dagen', yaxis_title='Frequentie', legend_title='Aanvragen', font=dict(size=12),
                  xaxis=dict(rangeslider={'visible': True}))
fig.show()

In [430]:
hover_data = {'ontvangst_oplevering':True, 'voorbereiding_overdracht':True, 'aanvraag_voorbereiding':True, 'aanvraag_oplevering':True, 'provincie':True}
fig = px.scatter_ternary(doorlooptijd_in_dagen, a="ontvangst_oplevering", b="voorbereiding_overdracht", c="aanvraag_voorbereiding", 
                         color="provincie",size='aanvraag_oplevering', size_max=20, opacity=0.8, height=600, width=800,
                         hover_data=hover_data)
fig.update_layout(title='Doorlooptijd', legend_title='Provincie', font=dict(size=12))
fig.update_traces(hovertemplate=
                  '''Provincie: %{customdata[4]}<extra></extra><br>Ontvangst aanvraag - werkvoorbereiding: %{customdata[2]} dagen<br>Werkvoorbereiding - overdracht aannemer: %{customdata[1]} dagen<br>Ontvangst aannemer - oplevering: %{customdata[0]} dagen<br>Totale doorlooptijd: %{customdata[3]} dagen'''
                )
fig.show()

In [415]:
eleknet = gpd.read_file('beschikbare_capaciteit_elektriciteitsnet.gpkg')
eleknet = eleknet.to_crs('EPSG:4326')

In [416]:
eleknet = eleknet.sjoin(prov, predicate = 'within')
eleknet.rename(columns = {'ligtInProvincieNaam':'provincie'}, inplace = True)

In [417]:
today = datetime.date.today()
year = today.year
eleknet['verwacht_jaar_van_overbelasting_afname'] = eleknet['verwacht_jaar_van_overbelasting_afname'].str.replace('>', '', regex=True)
eleknet['verwacht_jaar_van_overbelasting_afname'] = eleknet['verwacht_jaar_van_overbelasting_afname'].str.replace('[a-zA-Z]', '', regex=True)
eleknet['verwacht_jaar_van_overbelasting_afname'] = eleknet['verwacht_jaar_van_overbelasting_afname'].str.replace(' ', '', regex=True)
eleknet['verwacht_jaar_van_overbelasting_afname'] = pd.to_numeric(eleknet['verwacht_jaar_van_overbelasting_afname'])
eleknet['timedelta'] = (eleknet['verwacht_jaar_van_overbelasting_afname'] - year)

In [418]:
eleknet.head()

,station,netbeheerder,status,totale_capaciteit_invoeding_mva,totale_capaciteit_afname_mva,beschikbare_capaciteit_invoeding_huidig_mva,beschikbare_capaciteit_afname_huidig_mva,verwacht_jaar_van_overbelasting_invoeding,verwacht_jaar_van_overbelasting_afname,investeringsplanning_vermogen_invoeding,investeringsplanning_vermogen_afname,investeringsplanning_tijd_invoeding,investeringsplanning_tijd_afname,beschikbare_capaciteit_invoeding_3_jaar_mva,beschikbare_capaciteit_afname_3_jaar_mva,beschikbare_capaciteit_invoeding_5_jaar_mva,beschikbare_capaciteit_afname_5_jaar_mva,beschikbare_capaciteit_invoeding_10_jaar_mva,beschikbare_capaciteit_afname_10_jaar_mva,peildatum,beschikbare_capaciteit_afname_3_jaar_mva_visualisatie,beschikbare_capaciteit_afname_5_jaar_mva_visualisatie,beschikbare_capaciteit_afname_10_jaar_mva_visualisatie,beschikbare_capaciteit_afname_huidig_mva_visualisatie,beschikbare_capaciteit_invoeding_3_jaar_mva_visualisatie,beschikbare_capaciteit_invoeding_5_jaar_mva_visualisatie,beschikbare_capaciteit_invoeding_10_jaar_mva_visualisatie,beschikbare_capaciteit_invoeding_huidig_mva_visualisatie,geometry,index_right,provincie,timedelta
0,Krimpen_Langeland 50 kV,Stedin,Bestaand,117.0,117.0,142,12,2050,2026.0,onbekend,onbekend,2025,2025,-22,-117,-51,-123,-102,-140.5,2022/10/18,0,0,0,12,0,0,0,142,POINT (4.62915 51.91080),11,Zuid-Holland,3.0
1,Rotterdam Ommoord 25 kV,Stedin,Bestaand,115.0,115.0,157,-30,2050,2050.0,0,0,Geen investering gepland,Geen investering gepland,149,-26,148,-36,142,-66,2022/10/18,0,0,0,0,149,148,142,157,POINT (4.52947 51.95541),11,Zuid-Holland,27.0
2,Gouda Ijsseldijk 50 kV,Stedin,Bestaand,200.0,200.0,185,48,2045,2035.0,0,0,Geen investering gepland,Geen investering gepland,203,22,176,13,121,-30,2022/10/18,22,13,0,48,203,176,121,185,POINT (4.73411 52.00775),11,Zuid-Holland,12.0
3,Delft 1 25 kV,Stedin,Bestaand,187.0,187.0,205,46,2050,2027.0,0,0,Geen investering gepland,Geen investering gepland,201,15,191,4,159.5,-19,2022/10/18,15,4,0,46,201,191,160,205,POINT (4.36754 51.98904),11,Zuid-Holland,4.0
4,Grindweg 25 kV,Stedin,Bestaand,40.0,40.0,48,5,2050,2026.0,13,13,2025,2025,38,3,36,-1,30,-17.5,2022/10/18,3,0,0,5,38,36,30,48,POINT (4.49639 51.96228),11,Zuid-Holland,3.0


In [419]:
eleknet = eleknet.dropna()

In [420]:
eleknet['longitude'] = eleknet.geometry.x
eleknet['latitude'] = eleknet.geometry.y

In [421]:
px.set_mapbox_access_token('pk.eyJ1IjoieW9ycmlkIiwiYSI6ImNsZnV3cHllejAxazUzZ2x5Y25ua29jbnIifQ.DaEnA0YRKUq6q5eNbj-o0g')

In [425]:
hover_data = {'netbeheerder': False, 'verwacht_jaar_van_overbelasting_afname': True, 'timedelta': True, 'provincie':True, 'longitude': False, 'latitude': False}
fig = px.scatter_mapbox(eleknet, lat="latitude", lon="longitude", color="timedelta", 
                        color_continuous_scale='Viridis', size_max=15, zoom=6.2, labels={'timedelta':'Tijd (jaren)'},
                        hover_name='netbeheerder', title='Verwachte tijd voor overbelasting afname',
                        hover_data=hover_data, height=600, width=800, opacity=0.8)
fig.update_traces(hovertemplate=
                  '''Netbeheerder: %{hovertext}<extra></extra><br>Verwachte tijd tot overbelasting afname: %{customdata[2]} jaar<br>Provincie: %{customdata[3]}'''
                )
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(margin={"r":0, "t":60, "l":0,"b":0})
fig.show()